# Reduction traces of definable types

Our goal is to make pomagma.reducer smart enough to prove that I:UNIT,
as in definable_types.text (2016:08:23-25) (Q2):

<b>Desired Theorem:</b> `I : A \a,a',f,x. a(f(a' x))`, where

    copy := \x,y. x y y.
    join := \x,y,z. x(y|z).
    postconj := (\f. f \r,s. <B r, B s>).
    preconj := (\f. f \r,s. <CB s, CB r>).
    compose := (\f,f'. f\r,s. f'\r',s'. <r o r', s' o s>).
    A = A | <I, I> | <copy, join> | <div, BOT> | <BOT, TOP> | <C, C>
          | preconj A | postconj A | compose A A.

In [1]:
from pomagma.reducer import church
from pomagma.reducer.bohm import sexpr_simplify, try_compute_step
from pomagma.reducer.lib import BOT, TOP, B, C, I, box, pair
from pomagma.reducer.sugar import app, as_term, join_, rec
from pomagma.reducer.syntax import NVAR, sexpr_print

parse = sexpr_simplify


def pretty(term):
    if False:
        term = church.convert(term)
    return sexpr_print(term)

In [2]:
def trace(term, steps=10):
    '''Print a reduction sequence'''
    print('0 {}'.format(pretty(term)))
    for step in range(steps):
        term = try_compute_step(term)
        if term is None:
            print('[ Normalized ]')
            return
        print('{} {}'.format(1 + step, pretty(term)))
    print('[ Not Normalized ]')

In [3]:
CB = app(C, B)
div = rec(lambda a: join_(I, lambda x: app(a, x, TOP)))
copy = as_term(lambda x, y: app(x, y, y))
join = as_term(lambda x, y, z: app(x, join_(y, z)))
postconj = box(lambda r, s: pair(app(B, r), app(B, s)))
preconj = box(lambda r, s: pair(app(CB, s), app(CB, r)))
compose = as_term(
    lambda f1, f2: app(
        f1, lambda r1, s1: app(f2, lambda r2, s2: pair(app(B, r1, r2), app(B, s2, s1)))
    )
)

x, y, z = map(NVAR, "xyz")

Now we'll define the parts of `A`:
```
A = A | <I, I> | <copy, join> | <div, BOT> | <BOT, TOP> | <C, C>
      | preconj A | postconj A | compose A A.
```

In [4]:
PARTS = {
    "base": as_term(lambda a: pair(I, I)),
    "copy": as_term(lambda a: pair(copy, join)),
    "div": as_term(lambda a: pair(div, BOT)),
    "bot": as_term(lambda a: pair(BOT, TOP)),
    "swap": as_term(lambda a: pair(C, C)),
    "preconj": preconj,
    "postconj": postconj,
    "compose": as_term(lambda a: app(compose, a, a)),
}

In [5]:
def build_A(part_names):
    return rec(join_(*(PARTS[name] for name in part_names)))

In [6]:
default_type = parse("(FUN r (FUN s (FUN f (FUN x (r (f (s x)))))))")
default_inhab = parse("(FUN x x)")


def trace_A(part_names, steps=10, type_=default_type, inhab=default_inhab):
    # Cast a candidate inhabitant via a defined type.
    A = build_A(part_names)
    print("A = {}".format(pretty(A)))
    term = app(A, type_, inhab)
    trace(term, steps=steps)

In [7]:
trace_A(["preconj"], 2)

A = (ABS (0 0) (ABS (0 0 (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))))) (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))))
0 (ABS (0 0) (ABS (0 0 (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))))) (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))) (ABS (ABS (ABS (ABS (3 (1 (2 0))))))) (ABS 0))
1 (ABS (0 0) (ABS (0 0 (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))))) (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))) (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))) (ABS (ABS (ABS (ABS (3 (1 (2 0))))))) (ABS 0))
2 (ABS (0 0) (ABS (0 0 (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))))) (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))) (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))) (ABS (ABS (ABS (0 (ABS (ABS (1 (3 0)))) (ABS (ABS (1 (4 0)))))))) (ABS (ABS (ABS (ABS (3 (1 (2 0))))))) (ABS 0))
[ Not Normalized ]


Yuck, this is too difficult to read. We need better pretty printing tools.